In [19]:
WINDOW_SIZE = 300  # 3 second of data (300 * 10ms)
STEP_SIZE = 50     # Slide window by 500ms for more training samples
RANDOM_SEED = 42

In [20]:
import pandas as pd

def load_csv(filename):
    df = pd.read_csv(filename)
    return df['intensity'].values.astype('float32')

background = load_csv('/content/data/background.csv')
distractor = load_csv('/content/data/distractor.csv')
target = load_csv('/content/data/target.csv')

print(f"Loaded {len(background)} background samples")
print(f"Loaded {len(distractor)} distractor samples")
print(f"Loaded {len(target)} target samples")

Loaded 1082 background samples
Loaded 1082 distractor samples
Loaded 12756 target samples


In [21]:
import numpy as np

def create_windows(data, label, window_size, step):
    windows = []
    for i in range(0, len(data) - window_size, step):
        windows.append(data[i:i+window_size])
    return np.array(windows), np.array([label] * len(windows))

# Negative, no pests
x_bg, y_bg = create_windows(background, 0, WINDOW_SIZE, STEP_SIZE)
x_dist, y_dist = create_windows(distractor, 0, WINDOW_SIZE, STEP_SIZE)

# Raw target, pest sounds
x_target_clean, y_target_clean = create_windows(target, 1, WINDOW_SIZE, STEP_SIZE)

# Mix target and background
x_target_noisy = []
for win in x_target_clean:
    # Pick a random slice of background to overlay
    start = np.random.randint(0, len(distractor) - WINDOW_SIZE)
    dist_slice = distractor[start:start+WINDOW_SIZE]

    # Mix (pests + 30% Footstep)
    mixed = np.clip(win + (dist_slice * 0.3), 0, 100)
    x_target_noisy.append(mixed)

x_target_noisy = np.array(x_target_noisy)
y_target_noisy = np.array([1] * len(x_target_noisy))

# Combine everything
X = np.concatenate([x_bg, x_dist, x_target_clean, x_target_noisy])
y = np.concatenate([y_bg, y_dist, y_target_clean, y_target_noisy])

print(f"Total dataset size: {len(X)} samples")

Total dataset size: 532 samples


In [22]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

# Normalize to 0.0 - 1.0
X_train = X_train / 100.0
X_test = X_test / 100.0

In [23]:
from sklearn.utils import class_weight
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(WINDOW_SIZE,)),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid') # probability 0 to 1
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weights = {i: weights[i] for i in range(len(weights))}

# Add extra weight to silence to force it to zero
# This makes silence two times more important than pest sounds
class_weights[0] = class_weights[0] * 2.0 

history = model.fit(
    X_train, y_train, 
    epochs=50, 
    batch_size=16, 
    validation_data=(X_test, y_test),
    class_weight=class_weights
)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 16)             │         4,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,961 (19.38 KB)

 Trainable params: 4,961 (19.38 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 3s 51ms/step - accuracy: 0.9649 - loss: 0.9318 - val_accuracy: 0.9720 - val_loss: 0.4778
Epoch 2/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9699 - loss: 0.7314 - val_accuracy: 0.9720 - val_loss: 0.3749
Epoch 3/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9595 - loss: 0.7157 - val_accuracy: 0.9626 - val_loss: 0.3221
Epoch 4/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9628 - loss: 0.8394 - val_accuracy: 0.9626 - val_loss: 0.2788
Epoch 5/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9661 - loss: 0.6808 - val_accuracy: 0.9626 - val_loss: 0.2439
Epoch 6/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9610 - loss: 0.9591 - val_accuracy: 0.9626 - val_loss: 0.2130
Epoch 7/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9658 - loss: 0.7642 - val_accuracy: 0.9720 - val_loss: 0.1927
Epoch 8/50
27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9625 - loss: 0.8193 - val_accuracy: 0.9626 - val_loss

In [24]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

print(f"Model size: {len(tflite_model)} bytes")

Saved artifact at '/tmp/tmpp9xido0x'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 300), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  139929635369616: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139929635372112: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139929635371344: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139929635369424: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139929635372496: TensorSpec(shape=(), dtype=tf.resource, name=None)
  139929635372880: TensorSpec(shape=(), dtype=tf.resource, name=None)
Model size: 21912 bytes


In [ ]:
def hex_to_c_array(hex_data, var_name):
    c_str = f"const unsigned char {var_name}[] alignas(16) = {{\n  "
    for i, val in enumerate(hex_data):
        c_str += f"0x{val:02x}, "
        if (i + 1) % 12 == 0:
            c_str += "\n  "
    c_str += "\n};\n"
    c_str += f"const unsigned int {var_name}Len = {len(hex_data)};\n"
    return c_str

with open("/content/src/modes/PestModel.h", "w") as f:
    f.write(hex_to_c_array(tflite_model, "pestModelTflite"))

print("Saved PestModel.h")

Saved PestModel.h
